<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Module10_Unsupervised_upd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Learning: Clustering and PCA Analysis
Dani Bauer, 2023

As mentioned in the beginning of lecture, there are (at least) two basic learning setups.  In **supervised** machine learning one observes a response $Y$ with observing $p$ different features $X=(X_1,X_2,\ldots,X_p)$, where we typically postulate the relationship $Y = f(X)+\varepsilon$ and $\varepsilon$ independent of $X$ with mean zero. Here quality is usually assessed by the (test/out-of-sample) error that compares predictions and realizations for a separate dataset.  In **unsupervised** learning, we only observe $p$ features $X_1,X_2,\ldots,X_p$, and we would like to learn about their relationship -- without focussing on a supervising outcome.  Of course, the difficulty is how to assess quality in this case -- so different unsupervised learning techniques are very different, and which one to pick will depend on the nature of the problem.

In this tutorial, we will take a closer look at two algorithms: **Principal Component Analysis (PCA)** and **Clustering**.  There are variety of other techniques, including anomaly detection, self-organizing maps, association analysis, etc. We start with quick introductions but then look into the applications of these techniques in more detail in the context of a case study.

As usual, we start by implementing the relevant packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import plotly.figure_factory as ff
import pandas as pd
import seaborn as sns
from random import sample

from sklearn.preprocessing import MinMaxScaler, StandardScaler # For rescaling metrics to fit 0 to 1 range
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Principal Component Analysis

## Background

The key idea behind *Principal Component Analysis* (PCA) is to use "meaningful" linear combinations of the data:
$$
Z_m = \sum_{j=1}^p \phi_{jm} X_j \text{ such that }\sum_{j=1}^p \phi^2_{jm} = 1,
$$
where the idea is to choose:

- $\phi_{j1},\,j=1,\ldots,p$  such that the the variance of $Z_1$ is maximal (i.e., so that it captures most of the variation in the $X$s)

- $\phi_{j2},\,j=1,\ldots,p$ such that the variance of $Z_2$ is maximial out of all the variables that are uncorrelated with $Z_1$.

- $\phi_{j3},\,j=1,\ldots,p$ such that the variance of $Z_3$ is maximial out of all the variables that are uncorrelated with $Z_1$ and $Z_2$. $\ldots$

Hence, one way to intepret the principal components are the linear combination that best reflect the variation in the data.

Importantly, the scale of variables matters, so it is a good idea to center and scale your variables.  Also, the components are only determined up to their sign (plus/minus).  To discern how important different variables are, one typically considers the *Percent of Variance Explained* (PVE):
$$
\text{PVE}_m = \frac{\sum_{i=1}^n \left(\sum_{j=1}^p \phi_{jm}x_{ij}\right)^2}{\sum_{i=1}^n \sum_{j=1}^p x_{ij}^2},
$$
and the resulting plot that depicts PVE by principal components is referred to as a *scee plot*.

## Simulated Example

Let's consider a basic example, where we simulate heights and weights of a fictional population according to some arbitrary parameters.  More precisely, we assume that weight and height follow Normal distributions with a mean weight (in kg) of 70 and a mean height (in cm) of 170m, and variances of 25 and 150, respectively.  We assume the correlation parameter is 50%.


In [2]:
mu = (70,170)
cov = np.array([[25, np.sqrt(25)*np.sqrt(150)*0.5], [np.sqrt(25)*np.sqrt(150)*0.5, 150]])
X_raw = np.random.multivariate_normal(mu, cov, size=5000)

Let's check quick:

In [ ]:
X_raw.mean(axis=0)

In [ ]:
np.cov(np.transpose(X_raw))

In [ ]:
np.corrcoef(np.transpose(X_raw))

So the *empirical* moments look similar to our theoretical moments. Let's take a peak:


In [ ]:
plt.figure(figsize = (6,4))
plt.scatter(X_raw[:,0], X_raw[:,1], s = 1, color = 'black')
plt.xlabel('weight')
plt.ylabel('height')
plt.legend()
plt.show()

Prinicipal components are closely related to the eigen-analysis (eigenvalues and eigenvectors) of the correlation (or covariance) matrix.  Let's illustrate (if you never had a linear algebra class, feel free to skip this part).  First, let's scale the data and calculate the (empirical) correlation matrix -- remember, in practical settings we usually don't know the underlying parameters:


In [ ]:
R = np.corrcoef(np.transpose(X_raw))
R

Let's calculate the eigenvalues and the eigenvectors of `R`.  As a reminder, the eigenvectors decompose a matrix (a.k.a. a linear mapping in finite dimensions) in orthogonal directions, whereas the eignvalues provide the "length" (importance) of the directions.  In particular, we can represent a symmetric matrix in diagnolized form by relying on eigenvectors and eigenvalues.

In [ ]:
Dec = np.linalg.eig(R)
Dec

In [ ]:
V = Dec[1] # Matrix of Eigenvectores
D = np.diag(Dec[0]) # Diagonal Matrix of Eigenvalues
np.dot(np.dot(V,D),np.transpose(V)) # Calculates V * D * V'

To get intuition, let's plot the Eigenvectors:

In [ ]:
scaler = StandardScaler()
scaler.fit(X_raw)
plt.figure(figsize = (6,4))
fig, ax = plt.subplots()
ax.scatter(scaler.transform(X_raw)[:,0], scaler.transform(X_raw)[:,1], s = 1, color = 'black')
line1 = mlines.Line2D([0, 0.7071], [0, -.7071], color='red')
line2 = mlines.Line2D([0, .7071], [0, .7071], color='yellow')
transform = ax.transAxes
ax.add_line(line1)
ax.add_line(line2)
plt.xlabel('scaled weight')
plt.ylabel('scaled height')
plt.show()

Let's determine the PCAs to compare:

In [11]:
pca = PCA()
pca_fit = pca.fit(scaler.transform(X_raw))

In [ ]:
pca_fit.components_

It is exactly this notion of "importance" that motivates principal components.  Indeed, the loadings of the principal components just amount to the *ordered* eigenvalues...

# Clustering

## Background

*Clustering* refers to techniques for finding subgroups in a given dataset. The typical approach to determine clusters $C_1,\ldots,C_K$ is to minimize:
$$
\sum_{k=1}^K W(C_k),
$$
where $W$ is a measure of *variation* within a cluster.  For instance, **k-means clustering** uses the Euclidean distance to measure variation:
$$
W(C_k) = \frac{1}{|C_k|} \sum_{i,i' \in C_k} \sum_{j=1}^p (x_{ij} - x_{i'j})^2.
$$
The algorithms are implemented via a greedy algorithm by considering the centers of clusters (referred to as *centroids*).  The number of clusters $K$ must be chosen beforehand.  One approach is *hierarchical clustering*, where one starts with a larger number of clusters and then *fuses* custers that are similar (e.g., with regards to the distance between their centroids).

## Simulated Example

Let's consider a very basic simulated example -- let's simulate normal random variables with different means:

In [13]:
X_raw2 = np.random.multivariate_normal((0,0), np.array([[1, 0], [0, 1]]), size=100)
X_raw2[0:49,0]=X_raw2[1:50,0]+3
X_raw2[0:49,1]=X_raw2[1:50,1]-4

Let's plot:

In [ ]:
plt.figure(figsize = (6,4))
plt.scatter(X_raw2[0:49,0], X_raw2[0:49,1], color='red')
plt.scatter(X_raw2[50:99,0], X_raw2[50:99,1], color='black')
plt.xlabel('X0')
plt.ylabel('X1')
plt.legend()
plt.show()

And let's run k means clustering:


In [15]:
kmeans = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 1000, random_state = 123)
kmeans.fit(X_raw2)
centroids = kmeans.cluster_centers_
centroids

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([[ 0.21092013,  0.05185911],
       [ 2.89725937, -4.06564021]])

In [ ]:
label = kmeans.fit_predict(X_raw2)
label

So the algorithm was able to identify how we set up the data!

# Case Study: County Health Rankings 2013

We analyze [County Health Rankings](www.countyhealthrankings.org) in the US in 2013, based on a data set from the University of Wisconsin Population Health Institute.  I took this exammple from Jim Guszcza, and he put this together in R (it's much better than what I had before :-).

## Data Preparation

Let's load the data:

In [ ]:
!git clone https://github.com/danielbauer1979/ML_656.git

In [18]:
health = pd.read_csv('ML_656/countyHealthRR.csv')

In [ ]:
health.info()

So it's a large dataset, and the first three numbers are indices.  However, the remaning columns provide rather detailed information for each county.  Let's visualize:

In [ ]:
sns.heatmap(health.corr());

So quite a bit of correlation, but nothing seems to be "replicated."  

Unfortunately, we have a bunch of missing data. Let's drop the columns where we have lots of missing values and then drop na-s:

In [21]:
health = health.drop(columns=['FIPS','State','County','Perc.Fair.Poor.Health','Perc.Smokers','Perc.Excessive.Drinking','MV.Mortality.Rate','Pr.Care.Physician.Ratio','Perc.No.Soc.Emo.Support'])

In [22]:
health = health.dropna()

Let's look at the resulting data:

In [ ]:
health.info()

In [ ]:
health.describe()

And let's scale the data so as to make sure we are comparing apples to apples:

In [ ]:
scaler = MinMaxScaler()
scaler.fit(health)
health_sc = scaler.transform(health)
health_sc

## Clustering

### K-Means clustering

Let's commence with k-means clustering, which is avalable in the library `sklearn.cluster` (along with other clustering algorithms, see below). To decide on the number of clusters, we evaluate how the within-sum-of-squares varies between the number of clusters using a so-called *elbow plot*:


In [ ]:
wcss = []
for i in range(2, 12):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=1000, n_init=10, random_state=0)
    kmeans.fit(health_sc)
    wcss.append(kmeans.inertia_)
plt.bar(range(2, 12), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

It appears that six clusters seem to present a reasonable choice, there is a bit of an "elbow" there. So let's go with six:

In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', max_iter=1000, n_init=10, random_state=0)
kmeans.fit(health_sc)

We can get the labels via:

In [ ]:
kmeans.labels_

Let's add to our dataset:

In [29]:
health['k_means_cl'] = kmeans.labels_

Let's compare the different clusters by evaluating the YPLL.Rate (Years of Potential Life Lost).

In [ ]:
health[health['k_means_cl'] == 0]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 1]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 2]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 3]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 4]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 5]['YPLL.Rate'].mean()

Let's relabel so it's increasing:

In [36]:
def relab(label):
  if label == 0:
    return 0
  elif label == 1:
    return 1
  elif label == 2:
    return 5
  elif label == 3:
    return 2
  elif label == 4:
    return 4
  else:
    return 3

In [37]:
kmlab = list(map(relab, kmeans.labels_))

In [38]:
health['k_means_cl'] = kmlab

### Hierarchical Clustering

Let's use the hierarchical clustering algorithm in scikit (`AgglomerativeClustering`). Plotting a *dendrogram* is easier with the `scipy.cluster.hierarchy`, yet given the size of the dataset we don't see a ton here.

To compare with the k-means approach, let's run a hierarchical clustering also with six clusters.

In [ ]:
hierarch = AgglomerativeClustering(n_clusters=6)
hierarch.fit(health_sc)

Again, we can get the labels via:

In [ ]:
hierarch.labels_

Let's also add to the dataset and carry out a similar approach as before:

In [41]:
health['k_hier_cl'] = hierarch.labels_

In [ ]:
health[health['k_hier_cl'] == 0]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 1]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 2]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 3]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 4]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 5]['YPLL.Rate'].mean()

In [48]:
def relab2(label):
  if label == 0:
    return 5
  elif label == 1:
    return 2
  elif label == 2:
    return 0
  elif label == 3:
    return 4
  elif label == 4:
    return 3
  else:
    return 1

In [49]:
hierlab = list(map(relab2, hierarch.labels_))

In [50]:
health['k_hier_cl'] = hierlab

Let's compare the two:

In [ ]:
labels = [0,1,2,3,4,5]
confusion_matrix(health['k_means_cl'],health['k_hier_cl'],labels=labels)

So it looks like there is quite a bit of overlap, particularly with regards to clusters 0, 1, and 2. It appears clusters 4 and 5 are overlapping. However, it also is clear that assigning points to clusters isn't highly obvious.

### Visualizing Clusters

Let's visualize clusters by comparing the distributions of some of the features across clusters. As we had discussed, comparing distributions can be conveniently accomplished via box-whisker plots.

Let's start with children in poverty.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Perc.Children.in.Poverty", data = health)
plt.show()

Recall that we sorted the clusters according to YPLL.Rate (Years of Potential Life Lost), so more years of life lost is a sign of worse health. It appears that this weakly aligns with children poverty.

Let's look at violent crimes.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Violent.Crime.Rate", data = health)
plt.show()

So here we have a weak alignment, although cluster 4 seems a bit lower. Let's look at fast food concentration.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Perc.Fast.Foods", data = health)
plt.show()

### PCA

Recall that the dataset contains 24 features. As we explained above, PCA tries to determine the most relevant "directions" to span this space. Here, a "direction" corresponds to some weighted average of the data. The largest principal component is the most important direction, the second largest principal component is the second most important direction, etc.

We want to choose the number of principal components that accounts for a lot of variation in terms of "explained variance".

In [ ]:
plt.figure(figsize=(6,4))
pca = PCA(n_components=10)
pca_fit = pca.fit(health_sc)
pc_values = np.arange(pca_fit.n_components_) + 1
plt.bar(pc_values, pca.explained_variance_ratio_)
plt.title('Principal Component Analysis Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

Hence, the first three components explain:

In [56]:
pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1]+pca.explained_variance_ratio_[2]

0.5512169495805175

of the variance in the data. So while there is still a lot of other variation, the first three principal components explain a lot already.

So let's work with those:

In [ ]:
pca = PCA(n_components = 4)
principalComponents = pca.fit(health_sc)
principalComponents.components_
PCweights = pd.DataFrame(data = principalComponents.components_, columns = ["YPLL.Rate","Physically.Unhealthy.Days","Mentally.Unhealthy.Days","Perc.Low.Birth.Weight","Perc.Obese","Perc.Physically.Inactive","Chlamydia.Rate","Teen.Birth.Rate","Perc.Uninsured","Dentist.Ratio","Prev.Hosp.Stay.Rate","Perc.Diabetic.Screening","Perc.Mammography","Perc.High.School.Grad","Perc.Some.College","Perc.Unemployed","Perc.Children.in.Poverty","Perc.Single.Parent.HH","Violent.Crime.Rate","Avg.Daily.Particulates","Perc.pop.in.viol","Rec.Facility.Rate","Perc.Limited.Access","Perc.Fast.Foods"])
PCweights

Here we see that each component corresponds to the weight it puts on each of the 24 features. We can see that the first component puts about equal weight to the first six features, which were all signs of "bad health" (Physically.Unhealthy.Days, say). In contrast, the values of features 12 through 15 are negative, and these are all associated with "good health habits" (Perc.Diabetic.Screening, say). So scoring "high" on the first principal component (if the weighted average of features is "high") means that county is pretty unhealthy, ponentially due to funding or other aspects.

Let's look at the scores by county:

In [ ]:
principalComponentScores = pca.fit_transform(health_sc)
pc_scores = pd.DataFrame(data = principalComponentScores, columns = ['PC1', 'PC2','PC3','PC4'])
pc_scores

And let's add these to our data:

In [59]:
health = health.reset_index()

In [60]:
health = pd.concat([health,pc_scores], axis=1)

### PCA and Clusters

To intepret the connection of our clusters and the PC-s, let's check how the clusters relate to our first principal component.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "PC1", data = health)
plt.show()

This implies overall that cluster 0, 2, and 3 score low on PC1. Arguably these are out "low health" counties, which may have to do with infrastructure and how wealthy these counties are. Indeed, let's check the proportion of children in poverty:

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Perc.Children.in.Poverty", data = health)
plt.show()

This seems to track pretty well with the first principal component.

Let's look at the second principal component:

In [ ]:
sns.boxplot(x = "k_means_cl", y = "PC2", data = health)
plt.show()

So we see here the picture looks quite different -- clusters 1 and 4 score low on PCA2. This appears to some extent to connect to pollution, let's check particulates:

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Avg.Daily.Particulates", data = health)
plt.show()

So the counties in cluster 1 and 4 seem to have more polution.

To have a more formal comparison of clusters and principal components, let's prepare a scatterplot between PCs 1 and 2, and let's color them according to the clusters:

In [ ]:
groups = health.groupby("k_means_cl")
for name, group in groups:
    plt.plot(group.PC1, group.PC2, marker='o', linestyle='', markersize=4, label=name)
plt.title("Scatter plot between PC1 and PC2")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

So it's pretty clear that PC1 and PC2 do a good job at "explaining" the cluster labels.